In [281]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer



In [282]:
df2 = pd.read_csv('/Users/sanyakhan/Downloads/df_dropped_rows_w_statepartyna.csv')
df_unique = df2.drop_duplicates(subset=["Title"]).reset_index(drop=True)

In [283]:
custom = [ "agency",
"the","a","an","and","or","if","of","for","in","on","at","by","to","from","with", "people", "state", "want", "going",
"is","are","was","were","be","been","being","as","it","its","this","that","these","those",
"i","me","my","we","our","you","your","he","she","they","them","their","his","her", 'objection', 'rollcall',
"not","but","do","does","did","so","because","can","could","should","would","will","may","might",
"also","just","very","much","more","such","than","then","there","here","when","where","who","what","which","how",
"all","any","each","every","some","no","nor","only","own","same","too","into","up","down","about","over","under",
"thank","thankyou","yield","gentleman","gentlewoman","chair","chairman","chairwoman","speaker","presiding",
"recognize","recognized","recognition","remarks","floor","colleague","colleagues","proceed","order","ordered",
"committee","subcommittee","motion","unanimous","consent","debate","amendment","amend","section","title",
"chapter","paragraph","subparagraph","clause","statutory","herein","whereas","therefore","resolved","pursuant",
"enact","enacted","mr","mrs","ms","madam","sir","act","bill","resolution","senate","house","representative",
"representatives","congress","member","members","session","hearing","witness","testimony","law","legislation",
"legislative","vote","enactment","proceedings","statute","appropriation","federal","agency","program",
"department","secretary","policy","administration","executive","authority","director","office","ranking",
"majority","minority","sec","subsec","subtitle","appendix","figure","table","page","pages","line","lines",
"insert","strike","sentence","word","words","heading","part","division","codified","usc","code","public","law",
"number","numbers","date","dates","document","doc","clerk","shall","must","make","made","makes","take","taken",
"took","get","got","getting","give","gave","let","say","said","says","see","seen","look","looked","know","known",
"think","thought","myself","ours","ourselves","your","yours","yourself","yourselves","him","his","himself",
"hers","herself","theirs","themselves","itself","one","ones","someone","anyone","everyone","noone","many",
"much","none","both","either","neither","while","though","although","however","therefore","through","across",
"around","between","within","without","such","as","now","today","time","times","year","years","day","days",
"long","short","new","old","first","second","third","next","last","us","including","need","percent","miss", 'suspend',
"dr","chairperson","president","gentlelady","gentlemen","lady","panel","rules","rule","laws","policies", 'pre html', 'html'
"reading","read","united","states","america","american","washington","dc","bureau","tomorrow","week","body","week", "gentlewoman","body","floor","colleague","colleagues", 'move', 'congressional', 'record', 'vol', "appropriation", "senator", 'fl', 'johnson', 'ga', 'tx', 'recorded', 'smith', 'ny', 'rise', 'nation', 'honor', 'great', 'revise extend',' ask revise', 'revise', 'extend', 'ask', 'purposes ask', 'ask immediate','rollcall', 'voted', 'present', 'present voted', 'ask', 'objection', 'reconsider', 'laid', 'tempore_pro' 'tempore_pro_extraneous','yeas', 'nays']


BIGRAM_PAIRS = {
    ("affordable","housing"),
    ("domestic","violence"),
    ("mental","health"),
    ("chronic","homelessness"),
    ("emergency","shelter"),
    ("permanent","supportive"),
    ("rapid","rehousing"),
    ("veteran","homeless"),
    ("veterans","homeless"),
    ("people","experiencing"), 
    ("experiencing","homelessness"),
    ("unsheltered","homelessness"),
    ("supportive","housing"),
    ("substance","abuse"),
    ("housing","first"),
    ("youth","homelessness"),
}


STOP = set(stopwords.words("english"))
WORD = re.compile(r"[A-Za-z]+")
STOP.update(custom)

In [284]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download("punkt")
nltk.download("stopwords")
base_stop = set(stopwords.words("english"))
custom_stop = set(custom)      # your giant list
STOP = base_stop.union(custom_stop)

def remove_stopwords(text):
    text = str(text).lower()
    tokens = word_tokenize(text)
    cleaned = [t for t in tokens if t.isalpha() and t not in STOP]
    return " ".join(cleaned)

df_unique["clean_text"] = df_unique["speech_text"].apply(remove_stopwords)


#just cheking 
print(df_unique[["speech_text", "clean_text"]].head())


[nltk_data] Downloading package punkt to /Users/sanyakhan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sanyakhan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                         speech_text  \
0  Mr. Speaker, I move to suspend the rules and p...   
1  Madam Speaker, I move to suspend the rules and...   
2  Mr. Speaker, I move to suspend the rules and p...   
3  Mr. Speaker, I yield myself such time as I may...   
4  Mr. Speaker, I move to suspend the rules and p...   

                                          clean_text  
0  pass b housing exclude certain disability bene...  
1  pass authorize housing agencies share certain ...  
2  pass ensure children homeless veterans include...  
3  consume support homeless veterans assistance o...  
4  pass direct labor prioritize provision service...  


In [285]:
docs = df_unique["clean_text"].tolist()
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


umap_model = UMAP(
    n_neighbors=25,
    n_components=5,
    min_dist=0.1,
    metric="cosine",
    random_state=42,
)

#is this a good size 
hdbscan_model = HDBSCAN(
    min_cluster_size=10,
    min_samples=1,
    cluster_selection_epsilon=0.05,
    prediction_data=True,
)


vectorizer_model = CountVectorizer(
    stop_words=list(STOP),
    ngram_range=(1, 2),
    min_df=5,
    token_pattern=r"(?u)\b[A-Za-z][A-Za-z]+\b",
)

ctfidf_model = ClassTfidfTransformer(
    reduce_frequent_words=True
)

topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    calculate_probabilities=True,
    verbose=True,
)

topics, probs = topic_model.fit_transform(docs)

#maybe 20 for now
topic_model = topic_model.reduce_topics(docs, nr_topics=20)

df_unique["topic"] = topics
topic_info = topic_model.get_topic_info()
print(topic_info.head(20))
print("Number of outlier speeches:", (df_unique["topic"] == -1).sum())

2025-11-22 19:38:00,593 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/45 [00:00<?, ?it/s]

2025-11-22 19:38:15,243 - BERTopic - Embedding - Completed ✓
2025-11-22 19:38:15,249 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-22 19:38:19,433 - BERTopic - Dimensionality - Completed ✓
2025-11-22 19:38:19,435 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-22 19:38:19,575 - BERTopic - Cluster - Completed ✓
2025-11-22 19:38:19,582 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-22 19:38:21,721 - BERTopic - Representation - Completed ✓
2025-11-22 19:38:23,100 - BERTopic - Topic reduction - Reducing number of topics
2025-11-22 19:38:23,114 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-22 19:38:25,078 - BERTopic - Representation - Completed ✓
2025-11-22 19:38:25,080 - BERTopic - Topic reduction - Reduced number of topics from 45 to 20


    Topic  Count                                               Name  \
0      -1    367               -1_food_funds_available_agricultural   
1       0    224                        0_defense_military_air_army   
2       1    184      1_forfeiture_considered_consideration_printed   
3       2    170                  2_veterans_housing_health_affairs   
4       3     99               3_child_violence_trafficking_victims   
5       4     61  4_extraneous_extraneous material_material_include   
6       5     52  5_extraneous_extraneous material_tempore_pro t...   
7       6     38                      6_detained_res_votes_ordering   
8       7     35                7_museum_high school_defender_lives   
9       8     32  8_september purposes_fiscal ending_ending sept...   
10      9     25          9_minutes_divided_equally divided_equally   
11     10     24  10_statements relating_passed upon_statements_...   
12     11     19                  11_schools_students_http_teachers   
13    

In [286]:
df_unique[df_unique["topic"] == 3]

,Chamber,Year,Speaker,Party,State,Title,speech_text,turn_number,speech_length_words,clean_text,topic
0,House,2025,Hill,Republican,AR,HOUSING UNHOUSED DISABLED VETERANS ACT; Congre...,"Mr. Speaker, I move to suspend the rules and p...",1,422,pass b housing exclude certain disability bene...,3
1,House,2020,San Nicolas,Democrat,GU,HOMELESS ASSISTANCE ACT OF 2019; Congressional...,"Madam Speaker, I move to suspend the rules and...",1,402,pass authorize housing agencies share certain ...,3
2,House,2019,Takano,Democrat,CA,HOMELESS VETERAN FAMILIES ACT; Congressional R...,"Mr. Speaker, I move to suspend the rules and p...",1,234,pass ensure children homeless veterans include...,3
4,House,2021,Takano,Democrat,CA,HOMELESS VETERANS WITH CHILDREN REINTEGRATION ...,"Mr. Speaker, I move to suspend the rules and p...",1,313,pass direct labor prioritize provision service...,3
7,House,2019,Green,NaN,NaN,TRIBAL ACCESS TO HOMELESS ASSISTANCE ACT; Cong...,"Mr. Speaker, I move to suspend the rules and p...",1,450,pass homeless assistance enable indian tribes ...,3
8,House,2008,Moore,NaN,NaN,HOMELESS EMERGENCY ASSISTANCE AND RAPID TRANSI...,"Mr. Speaker, I move to suspend the rules and p...",1,16309,pass homeless assistance reauthorize purposes ...,3
22,House,2020,Takano,Democrat,CA,LEGAL SERVICES FOR HOMELESS VETERANS ACT; Cong...,"Madam Speaker, I move to suspend the rules and...",1,1018,pass direct veterans affairs grants entities p...,3
45,House,2020,Waters,Democrat,CA,EMERGENCY HOUSING PROTECTIONS AND RELIEF ACT O...,"Mr. Speaker, pursuant to House Resolution 1017...",1,14776,call prevent evictions foreclosures unsafe hou...,3
46,House,2020,Waters,Democrat,CA,VETERAN HOUSING OPPORTUNITIES AND UNEMPLOYMENT...,"Madam Speaker, I move to suspend the rules and...",1,433,pass housing expand eligibility direct veteran...,3
48,House,2020,Sherman,Democrat,CA,IMPROVING EMERGENCY DISEASE RESPONSE VIA HOUSI...,"Mr. Speaker, I move to suspend the rules and p...",1,320,pass require data sharing regarding protecting...,3


In [287]:
docs = df_unique["clean_text"].tolist()
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


umap_model = UMAP(
    n_neighbors=25,
    n_components=5,
    min_dist=0.1,
    metric="cosine",
    random_state=42,
)

hdbscan_model = HDBSCAN(
    min_cluster_size=10,
    min_samples=1,
    cluster_selection_epsilon=0.05,
    prediction_data=True,
)


vectorizer_model = CountVectorizer(
    stop_words=list(STOP),
    ngram_range=(1, 2),
    min_df=5,
    token_pattern=r"(?u)\b[A-Za-z][A-Za-z]+\b",
)

ctfidf_model = ClassTfidfTransformer(
    reduce_frequent_words=True
)

topic_model = BERTopic(
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,
    ctfidf_model=ctfidf_model,
    calculate_probabilities=True,
    verbose=True,
)

topics, probs = topic_model.fit_transform(docs)

topic_model = topic_model.reduce_topics(docs, nr_topics=20)

df_unique["topic"] = topics
topic_info = topic_model.get_topic_info()
print(topic_info.head(20))
print("Number of outlier speeches:", (df_unique["topic"] == -1).sum())

2025-11-22 19:38:31,320 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/45 [00:00<?, ?it/s]

2025-11-22 19:38:46,166 - BERTopic - Embedding - Completed ✓
2025-11-22 19:38:46,168 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-11-22 19:38:50,181 - BERTopic - Dimensionality - Completed ✓
2025-11-22 19:38:50,182 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-11-22 19:38:50,312 - BERTopic - Cluster - Completed ✓
2025-11-22 19:38:50,316 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-22 19:38:52,330 - BERTopic - Representation - Completed ✓
2025-11-22 19:38:53,690 - BERTopic - Topic reduction - Reducing number of topics
2025-11-22 19:38:53,694 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-11-22 19:38:55,608 - BERTopic - Representation - Completed ✓
2025-11-22 19:38:55,610 - BERTopic - Topic reduction - Reduced number of topics from 45 to 20


    Topic  Count                                               Name  \
0      -1    367               -1_food_funds_available_agricultural   
1       0    224                        0_defense_military_air_army   
2       1    184      1_forfeiture_considered_consideration_printed   
3       2    170                  2_veterans_housing_health_affairs   
4       3     99               3_child_violence_trafficking_victims   
5       4     61  4_extraneous_extraneous material_material_include   
6       5     52  5_extraneous_extraneous material_tempore_pro t...   
7       6     38                      6_detained_res_votes_ordering   
8       7     35                7_museum_high school_defender_lives   
9       8     32  8_september purposes_fiscal ending_ending sept...   
10      9     25          9_minutes_divided_equally divided_equally   
11     10     24  10_statements relating_passed upon_statements_...   
12     11     19                  11_schools_students_http_teachers   
13    